<a href="https://colab.research.google.com/github/pereira-71/PRACTICA-01-SIS421/blob/main/Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1: Instalación de dependencias

In [31]:
!pip install requests beautifulsoup4 lxml mutagen python-magic
print("Bibliotecas instaladas correctamente")

Bibliotecas instaladas correctamente


requests: Para hacer solicitudes HTTP

beautifulsoup4 y lxml: Para parsear HTML

mutagen: Para validar archivos de audio

python-magic: Para verificar tipos MIME reales de los archivos

2: Configuración inicial

In [32]:
import os
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import mimetypes
import magic
from mutagen.easyid3 import EasyID3
from mutagen.mp3 import MP3
from tqdm import tqdm

# Configuración global
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
AUDIO_EXTENSIONS = ['.mp3', '.wav', '.ogg', '.m4a', '.aac', '.flac', '.wma', '.mp4']
MAX_FILE_SIZE_MB = 50  # Límite de tamaño por archivo
REQUEST_TIMEOUT = 15  # segundos
DELAY_BETWEEN_DOWNLOADS = 1  # segundo

 3: Función principal de scraping

In [33]:
def descargar_audios(url_pagina, carpeta_destino="audios_descargados", max_audios=20):
    """
    Función optimizada para descargar archivos de audio de páginas web

    Args:
        url_pagina (str): URL de la página a scrapear
        carpeta_destino (str): Carpeta para guardar audios
        max_audios (int): Máximo número de audios a descargar (hasta 20)

    Returns:
        int: Número de audios descargados exitosamente
    """

    # Validación de parámetros
    max_audios = min(max(1, max_audios), 20)

    # Crear carpeta con timestamp para evitar conflictos
    timestamp = int(time.time())
    carpeta_destino = f"{carpeta_destino}_{timestamp}"
    os.makedirs(carpeta_destino, exist_ok=True)

    # Obtener contenido de la página
    try:
        print(f"\n Analizando página: {url_pagina}")
        response = requests.get(url_pagina, headers=HEADERS, timeout=REQUEST_TIMEOUT)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'lxml')
    except Exception as e:
        print(f" Error al acceder a la página: {str(e)}")
        return 0

    # Estrategias de búsqueda de audios
    sources = []

    # 1. Buscar etiquetas <audio> y sus <source>
    for audio in soup.find_all('audio'):
        if src := audio.get('src'):
            sources.append(src)
        for source in audio.find_all('source'):
            if src := source.get('src'):
                sources.append(src)

    # 2. Buscar enlaces con extensiones de audio
    for a in soup.find_all('a', href=True):
        href = a['href'].lower()
        if any(ext in href for ext in AUDIO_EXTENSIONS):
            sources.append(a['href'])

    # 3. Buscar en atributos data-*
    for tag in soup.find_all(attrs={"data-src": True}):
        sources.append(tag.get('data-src'))
    for tag in soup.find_all(attrs={"data-audio": True}):
        sources.append(tag.get('data-audio'))

    # Eliminar duplicados y URLs inválidas
    sources = list(set(filter(None, sources)))
    print(f"🔊 Se encontraron {len(sources)} posibles fuentes de audio")

    # Procesar las fuentes encontradas
    audios_descargados = 0
    progress_bar = tqdm(sources[:max_audios], desc="Descargando audios", unit="audio")

    for src in progress_bar:
        if audios_descargados >= max_audios:
            break

        try:
            audio_url = urljoin(url_pagina, src)
            progress_bar.set_postfix({"actual": audio_url.split('/')[-1][:20] + '...'})

            # Verificación previa
            head_resp = requests.head(audio_url, headers=HEADERS, timeout=REQUEST_TIMEOUT)
            content_type = head_resp.headers.get('content-type', '')
            content_length = int(head_resp.headers.get('content-length', 0))

            # Validar tipo y tamaño
            if not ('audio' in content_type.lower() or any(audio_url.lower().endswith(ext) for ext in AUDIO_EXTENSIONS)):
                continue

            if content_length > MAX_FILE_SIZE_MB * 1024 * 1024:
                print(f" Archivo demasiado grande: {audio_url}")
                continue

            # Descargar el archivo
            audio_resp = requests.get(audio_url, headers=HEADERS, timeout=REQUEST_TIMEOUT, stream=True)
            audio_resp.raise_for_status()

            # Determinar extensión
            ext = os.path.splitext(urlparse(audio_url).path)[1].lower()
            if not ext or ext not in AUDIO_EXTENSIONS:
                ext = mimetypes.guess_extension(content_type) or '.mp3'

            # Guardar archivo
            filename = f"audio_{audios_descargados+1}{ext}"
            filepath = os.path.join(carpeta_destino, filename)

            with open(filepath, 'wb') as f:
                for chunk in audio_resp.iter_content(chunk_size=8192):
                    f.write(chunk)

            # Validar que sea un audio real
            try:
                mime = magic.from_file(filepath, mime=True)
                if not mime.startswith('audio/'):
                    os.remove(filepath)
                    continue

                # Extraer metadatos si es MP3
                if filename.endswith('.mp3'):
                    try:
                        audio = MP3(filepath, ID3=EasyID3)
                        print(f"\n🎵 Metadatos: {audio.pprint()}")
                    except:
                        pass

                audios_descargados += 1
                print(f"OK Descargado: {filename} ({content_length/(1024*1024):.2f} MB)")

            except Exception as e:
                print(f"Error validando archivo: {str(e)}")
                if os.path.exists(filepath):
                    os.remove(filepath)

            time.sleep(DELAY_BETWEEN_DOWNLOADS)

        except Exception as e:
            print(f" Error procesando {src}: {str(e)}")
            continue

    print(f"\n Descarga completada! {audios_descargados} audios guardados en '{carpeta_destino}'")
    return audios_descargados

4: Ejecución del scraping

In [34]:
# URLs de prueba con contenido de audio
AUDIO_SAMPLES = [
    "https://www.soundhelix.com/audio-examples",
    "https://www.soundjay.com/button-sounds-1.html"

]

# Seleccionamos una URL para probar
url_seleccionada = AUDIO_SAMPLES[1]  # podemos cambiar índice para probar diferentes sitios

# Ejecutar scraping
resultado = descargar_audios(
    url_pagina=url_seleccionada,
    carpeta_destino="mis_audios",
    max_audios=20  # Máximo permitido para aprendizaje
)

print(f"\nTotal de audios descargados: {resultado}")


 Analizando página: https://www.soundjay.com/button-sounds-1.html
🔊 Se encontraron 30 posibles fuentes de audio


Descargando audios:   0%|          | 0/20 [00:00<?, ?audio/s, actual=button-8.mp3...]


🎵 Metadatos: MPEG 1 layer 3, 128000 bps (CBR?), 44100 Hz, 1 chn, 1.04 seconds (audio/mp3)
artist=SoundJay.com Sound Effects
OK Descargado: audio_1.mp3 (0.02 MB)


Descargando audios:   5%|▌         | 1/20 [00:01<00:21,  1.11s/audio, actual=button-09a.mp3...]


🎵 Metadatos: MPEG 1 layer 3, 128000 bps (CBR?), 44100 Hz, 1 chn, 0.47 seconds (audio/mp3)
artist=SoundJay.com Sound Effects
OK Descargado: audio_2.mp3 (0.01 MB)


...

OK Descargado: audio_19.wav (0.03 MB)


Descargando audios:  95%|█████████▌| 19/20 [00:20<00:01,  1.10s/audio, actual=button-4.mp3...]


🎵 Metadatos: MPEG 1 layer 3, 128000 bps (CBR?), 44100 Hz, 1 chn, 0.68 seconds (audio/mp3)
artist=SoundJay.com Sound Effects
OK Descargado: audio_20.mp3 (0.01 MB)


Descargando audios: 100%|██████████| 20/20 [00:21<00:00,  1.10s/audio, actual=button-4.mp3...]


 Descarga completada! 20 audios guardados en 'mis_audios_1754105763'

Total de audios descargados: 20


5: Reproducción y verificación

In [35]:
import os

# Encuentra todas las carpetas que empiecen por 'mis_audios'
carpetas = sorted([
    d for d in os.listdir()
    if os.path.isdir(d) and d.startswith("mis_audios_")
], key=lambda x: os.path.getmtime(x))

if carpetas:
    ultima_carpeta = carpetas[-1]
    print(f"🎧 Reproduciendo audios desde: {ultima_carpeta}")

    from IPython.display import Audio, display

    # Reproduce todos los audios encontrados
    for nombre in sorted(os.listdir(ultima_carpeta)):
        if nombre.lower().endswith(('.mp3', '.wav')):
            print(f"\n{name}")
            display(Audio(filename=os.path.join(ultima_carpeta, nombre)))
else:
    print("No se encontraron carpetas con audios")


🎧 Reproduciendo audios desde: mis_audios_1754105763


NameError: name 'name' is not defined